In [7]:
import json
import os
import sys
from collections import defaultdict
from typing import Dict, List
PROGRESS_INTERVAL = 10000 # 10,000개 샘플마다 진행 상황 출력
# --- 설정 ---
# JSON 파일 경로 (사용자 프로젝트의 50% 데이터 경로)
JSON_PATH = "/nas/datahub/llava-v1.5-instruct/llava_v1_5_mix665k_50pct.json"
# ----------------

def check_missing_images(json_path: str, image_root: str) -> Dict[str, List[str]]:
    """
    JSON 파일을 검사하여 image_root 경로에 이미지가 실제로 존재하는지 확인합니다.
    """
    if not os.path.exists(json_path):
        print(f"Error: JSON file not found at {json_path}", file=sys.stderr)
        return {}

    with open(json_path, 'r') as f:
        data = json.load(f)

    total_samples = len(data)
    # 딕셔너리의 키로 데이터셋 이름(vg, coco 등)을 사용하기 위해 defaultdict 사용
    missing_files: Dict[str, List[str]] = defaultdict(list)
    total_samples = 0
    missing_count = 0

    print(f"--- Starting validation: {len(data)} samples ---")
    
    for i, item in enumerate(data):
            
        # --- 진행 현황 출력 로직 (추가) ---
        if (i + 1) % PROGRESS_INTERVAL == 0:
            percent = (i + 1) / total_samples * 100
            print(f"--- Progress: {i + 1} / {total_samples} samples checked ({percent:.2f}%) ---")
            # ------------------------------------        
        total_samples += 1
        # 'image' 키가 없는 경우는 텍스트 전용 데이터이므로 건너뜀
        relative_path = item.get("image")
        
        if relative_path:
            # 1. 절대 경로 결합: image_root + relative_path
            full_path = os.path.join(image_root, relative_path)
            
            # 2. 파일 존재 여부 확인
            if not os.path.exists(full_path):
                missing_count += 1
                
                # 데이터셋 이름 추출 (예: 'vg/VG_100K/...' -> 'vg')
                # LLaVA JSON 경로는 리눅스 경로 구분자('/')를 사용한다고 가정
                dataset_name = relative_path.split(os.sep)[0]
                
                missing_files[dataset_name].append(relative_path)
                
                # 진행 상황을 터미널에 출력 (선택 사항)
                if missing_count % 100 == 0:
                    print(f"[{missing_count} missing found] Last checked: {full_path}")

    print(f"\n--- Validation Complete ---")
    print(f"Total samples checked: {total_samples}")
    print(f"Total missing files found: {missing_count}")
    return missing_files


In [8]:

if __name__ == "__main__":
    if len(sys.argv) != 2:
        print(f"Usage: python {sys.argv[0]} [ABSOLUTE_IMAGE_ROOT_PATH]")
        print("Example: python validate_dataset.py /nas/datahub/llava-v1.5-instruct/images")
        sys.exit(1)
    
    # --image_folder에 해당하는 절대 경로를 인수로 받음
    image_root_path = '/nas/datahub/llava-v1.5-instruct'
    
    results = check_missing_images(JSON_PATH, image_root_path)

    # --- 결과 출력 ---
    if results:
        print("\n\n❌ MISSING FILES REPORT ❌")
        for dataset, paths in results.items():
            print(f"\n[Dataset: {dataset.upper()}] - {len(paths)} missing files")
            # 누락된 파일 목록 중 처음 5개만 출력
            for path in paths[:5]:
                print(f"  - {path}")
            if len(paths) > 5:
                print(f"  ... and {len(paths) - 5} more.")
        
        # 전체 딕셔너리를 JSON 파일로 저장 (선택 사항)
        with open("missing_files_report.json", "w") as f:
            json.dump(results, f, indent=4)
        print("\nFull report saved to missing_files_report.json")
    else:
        print("\n✅ All image files found successfully! You can start training safely.")

--- Starting validation: 332649 samples ---
--- Progress: 10000 / 9999 samples checked (100.01%) ---
--- Progress: 20000 / 19999 samples checked (100.01%) ---
--- Progress: 30000 / 29999 samples checked (100.00%) ---
--- Progress: 40000 / 39999 samples checked (100.00%) ---
--- Progress: 50000 / 49999 samples checked (100.00%) ---
--- Progress: 60000 / 59999 samples checked (100.00%) ---
--- Progress: 70000 / 69999 samples checked (100.00%) ---
--- Progress: 80000 / 79999 samples checked (100.00%) ---
--- Progress: 90000 / 89999 samples checked (100.00%) ---
--- Progress: 100000 / 99999 samples checked (100.00%) ---
--- Progress: 110000 / 109999 samples checked (100.00%) ---
--- Progress: 120000 / 119999 samples checked (100.00%) ---
--- Progress: 130000 / 129999 samples checked (100.00%) ---
--- Progress: 140000 / 139999 samples checked (100.00%) ---
--- Progress: 150000 / 149999 samples checked (100.00%) ---
--- Progress: 160000 / 159999 samples checked (100.00%) ---
--- Progress: 17

In [1]:
import json
from PIL import Image
from pathlib import Path
from tqdm.notebook import tqdm

# 경로 설정
base_dir = Path("/nas/datahub/llava-v1.5-instruct")
ocr_vqa_dir = base_dir / "ocr_vqa/images"

# JSON 데이터 (제공하신 파일 리스트)
missing_jpg_list = {
    "ocr_vqa": [
        "ocr_vqa/images/761525904.jpg",
        "ocr_vqa/images/299130002.jpg",
        "ocr_vqa/images/785807209.jpg",
        "ocr_vqa/images/849930987.jpg",
        "ocr_vqa/images/679888268.jpg",
        "ocr_vqa/images/1574770225.jpg",
        "ocr_vqa/images/912111364.jpg",
        "ocr_vqa/images/870331612.jpg",
        "ocr_vqa/images/780800370.jpg",
        "ocr_vqa/images/087033512X.jpg",
        "ocr_vqa/images/312890583.jpg",
        "ocr_vqa/images/749517735.jpg",
        "ocr_vqa/images/1566250420.jpg",
        "ocr_vqa/images/786405511.jpg",
        "ocr_vqa/images/688118127.jpg",
        "ocr_vqa/images/934710171.jpg",
        "ocr_vqa/images/931580390.jpg",
        "ocr_vqa/images/739704133.jpg",
        "ocr_vqa/images/812931432.jpg",
        "ocr_vqa/images/139642625.jpg",
        "ocr_vqa/images/1580170536.jpg",
        "ocr_vqa/images/133099156.jpg",
        "ocr_vqa/images/670886939.jpg",
        "ocr_vqa/images/1560799722.jpg",
        "ocr_vqa/images/452255260.jpg",
        "ocr_vqa/images/717281434.jpg",
        "ocr_vqa/images/1885726031.jpg",
        "ocr_vqa/images/1557987203.jpg",
        "ocr_vqa/images/1579771009.jpg",
        "ocr_vqa/images/966355903.jpg",
        "ocr_vqa/images/688121675.jpg",
        "ocr_vqa/images/899330266.jpg",
        "ocr_vqa/images/078942049X.jpg",
        "ocr_vqa/images/067944680X.jpg",
        "ocr_vqa/images/898796482.jpg",
        "ocr_vqa/images/080740604X.jpg",
        "ocr_vqa/images/914625179.jpg",
        "ocr_vqa/images/134412052.jpg",
        "ocr_vqa/images/B013RVJ7KW.jpg",
        "ocr_vqa/images/1560445513.jpg",
        "ocr_vqa/images/316051772.jpg",
        "ocr_vqa/images/764504665.jpg",
        "ocr_vqa/images/3980621154.jpg",
        "ocr_vqa/images/870408712.jpg",
        "ocr_vqa/images/679844023.jpg",
        "ocr_vqa/images/1579901387.jpg",
        "ocr_vqa/images/812035712.jpg",
        "ocr_vqa/images/877936293.jpg",
        "ocr_vqa/images/968557902.jpg",
        "ocr_vqa/images/268016860.jpg",
        "ocr_vqa/images/1878239589.jpg",
        "ocr_vqa/images/810955563.jpg",
        "ocr_vqa/images/188673206X.jpg",
        "ocr_vqa/images/1572240466.jpg",
        "ocr_vqa/images/3797306210.jpg",
        "ocr_vqa/images/679445765.jpg",
        "ocr_vqa/images/051770353X.jpg",
        "ocr_vqa/images/688116191.jpg",
        "ocr_vqa/images/963479903.jpg",
        "ocr_vqa/images/29344506.jpg",
        "ocr_vqa/images/953735702.jpg",
        "ocr_vqa/images/706377273.jpg",
        "ocr_vqa/images/750219378.jpg",
        "ocr_vqa/images/1570611912.jpg",
        "ocr_vqa/images/1560443952.jpg",
        "ocr_vqa/images/377570177X.jpg",
        "ocr_vqa/images/2061514022.jpg",
        "ocr_vqa/images/B00XIZWWNC.jpg",
        "ocr_vqa/images/394532643.jpg",
        "ocr_vqa/images/1564583015.jpg",
        "ocr_vqa/images/1565042301.jpg",
        "ocr_vqa/images/1568590644.jpg",
        "ocr_vqa/images/878576991.jpg",
        "ocr_vqa/images/876043082.jpg",
        "ocr_vqa/images/157851231X.jpg",
        "ocr_vqa/images/1567615317.jpg",
        "ocr_vqa/images/3575228701.jpg",
        "ocr_vqa/images/807085707.jpg",
        "ocr_vqa/images/835607240.jpg",
        "ocr_vqa/images/1878239376.jpg",
        "ocr_vqa/images/1881174034.jpg",
        "ocr_vqa/images/570035651.jpg",
        "ocr_vqa/images/156458321X.jpg",
        "ocr_vqa/images/936783109.jpg",
        "ocr_vqa/images/B011M9LHUO.jpg",
        "ocr_vqa/images/879802863.jpg",
        "ocr_vqa/images/893148423.jpg",
        "ocr_vqa/images/1558743030.jpg",
        "ocr_vqa/images/812903390.jpg",
        "ocr_vqa/images/871563932.jpg",
        "ocr_vqa/images/812015320.jpg",
        "ocr_vqa/images/673384772.jpg",
        "ocr_vqa/images/942110439.jpg",
        "ocr_vqa/images/006270110X.jpg",
        "ocr_vqa/images/231072430.jpg",
        "ocr_vqa/images/1568811012.jpg",
        "ocr_vqa/images/471523771.jpg",
        "ocr_vqa/images/1564556409.jpg",
        "ocr_vqa/images/915801841.jpg",
        "ocr_vqa/images/878918515.jpg",
        "ocr_vqa/images/739714600.jpg",
        "ocr_vqa/images/28624084.jpg",
        "ocr_vqa/images/898796008.jpg",
        "ocr_vqa/images/28628594.jpg",
        "ocr_vqa/images/B00XLZW19O.jpg",
        "ocr_vqa/images/968297072.jpg",
        "ocr_vqa/images/531202852.jpg",
        "ocr_vqa/images/472084798.jpg",
        "ocr_vqa/images/135707978.jpg",
        "ocr_vqa/images/553353500.jpg",
        "ocr_vqa/images/078710339X.jpg",
        "ocr_vqa/images/750223391.jpg",
        "ocr_vqa/images/811820580.jpg",
        "ocr_vqa/images/789401509.jpg",
        "ocr_vqa/images/B001KBBD4A.jpg",
        "ocr_vqa/images/761500413.jpg",
        "ocr_vqa/images/185230863X.jpg",
        "ocr_vqa/images/060960323X.jpg",
        "ocr_vqa/images/590366742.jpg",
        "ocr_vqa/images/933478186.jpg",
        "ocr_vqa/images/1862045852.jpg",
        "ocr_vqa/images/1559920734.jpg",
        "ocr_vqa/images/823929493.jpg",
        "ocr_vqa/images/1570190380.jpg",
        "ocr_vqa/images/810940183.jpg",
        "ocr_vqa/images/785282394.jpg",
        "ocr_vqa/images/1570282358.jpg",
        "ocr_vqa/images/1890838004.jpg",
        "ocr_vqa/images/055305340X.jpg",
        "ocr_vqa/images/812035844.jpg",
        "ocr_vqa/images/966542606.jpg",
        "ocr_vqa/images/879514795.jpg",
        "ocr_vqa/images/810980789.jpg",
        "ocr_vqa/images/739704516.jpg",
        "ocr_vqa/images/1557488789.jpg",
        "ocr_vqa/images/78821185.jpg",
        "ocr_vqa/images/811726819.jpg",
        "ocr_vqa/images/819183482.jpg",
        "ocr_vqa/images/71351817.jpg",
        "ocr_vqa/images/093062596X.jpg",
        "ocr_vqa/images/739715534.jpg",
        "ocr_vqa/images/1887089160.jpg",
        "ocr_vqa/images/471243787.jpg",
        "ocr_vqa/images/1555951120.jpg",
        "ocr_vqa/images/1569750882.jpg",
        "ocr_vqa/images/415913756.jpg",
        "ocr_vqa/images/3980621146.jpg",
        "ocr_vqa/images/832904651.jpg",
        "ocr_vqa/images/1883323703.jpg",
        "ocr_vqa/images/525941290.jpg",
        "ocr_vqa/images/471550833.jpg",
        "ocr_vqa/images/1886947694.jpg",
        "ocr_vqa/images/761522751.jpg",
        "ocr_vqa/images/201112973.jpg",
        "ocr_vqa/images/375400664.jpg",
        "ocr_vqa/images/1556226535.jpg",
        "ocr_vqa/images/1568362021.jpg",
        "ocr_vqa/images/393098745.jpg",
        "ocr_vqa/images/60191341.jpg",
        "ocr_vqa/images/1561701289.jpg",
        "ocr_vqa/images/312187114.jpg",
        "ocr_vqa/images/879801654.jpg",
        "ocr_vqa/images/1555951295.jpg",
        "ocr_vqa/images/961742607.jpg",
        "ocr_vqa/images/821411896.jpg",
        "ocr_vqa/images/205260780.jpg",
        "ocr_vqa/images/789401592.jpg",
        "ocr_vqa/images/674035275.jpg",
        "ocr_vqa/images/B00XLX3W9O.jpg",
        "ocr_vqa/images/471148288.jpg",
        "ocr_vqa/images/891960856.jpg",
        "ocr_vqa/images/1563053128.jpg",
        "ocr_vqa/images/960536205.jpg",
        "ocr_vqa/images/967008808.jpg",
        "ocr_vqa/images/B005FOFNA8.jpg",
        "ocr_vqa/images/1574301012.jpg"
    ]
}

# 또는 파일명만 추출
file_names = [Path(p).name for p in missing_jpg_list["ocr_vqa"]]

print(f"Total files to convert: {len(file_names)}")
print(f"OCR-VQA directory: {ocr_vqa_dir}")
print()

# GIF → JPG 변환 함수
def convert_gif_to_jpg(gif_path, jpg_path, quality=95):
    """GIF를 JPG로 변환"""
    try:
        # GIF 열기
        img = Image.open(gif_path)
        
        # RGB 변환 (GIF는 P, RGBA 등일 수 있음)
        if img.mode != 'RGB':
            if img.mode in ('RGBA', 'LA', 'P'):
                # 흰색 배경 생성
                background = Image.new('RGB', img.size, (255, 255, 255))
                
                # P 모드는 RGBA로 먼저 변환
                if img.mode == 'P':
                    img = img.convert('RGBA')
                
                # Alpha 채널이 있으면 마스크로 사용
                if img.mode == 'RGBA':
                    background.paste(img, mask=img.split()[-1])
                else:
                    background.paste(img)
                
                img = background
            else:
                img = img.convert('RGB')
        
        # JPG로 저장
        img.save(jpg_path, 'JPEG', quality=quality)
        return True, None
        
    except Exception as e:
        return False, str(e)

# 변환 실행
print("=" * 70)
print("Converting GIF to JPG")
print("=" * 70)

success_count = 0
already_exists = 0
gif_not_found = 0
conversion_failed = []
no_gif_alternative = []

for jpg_name in tqdm(file_names, desc="Processing"):
    jpg_path = ocr_vqa_dir / jpg_name
    
    # JPG가 이미 있으면 스킵
    if jpg_path.exists():
        already_exists += 1
        continue
    
    # GIF 경로 확인
    gif_path = jpg_path.with_suffix('.gif')
    
    if gif_path.exists():
        # GIF → JPG 변환
        success, error = convert_gif_to_jpg(gif_path, jpg_path)
        
        if success:
            success_count += 1
        else:
            conversion_failed.append({
                'file': jpg_name,
                'error': error
            })
    else:
        # GIF도 없는 경우 - 다른 확장자 확인
        found_alternative = False
        for ext in ['.png', '.jpeg', '.PNG', '.GIF', '.JPEG']:
            alt_path = jpg_path.with_suffix(ext)
            if alt_path.exists():
                # 다른 확장자를 JPG로 변환
                success, error = convert_gif_to_jpg(alt_path, jpg_path)
                if success:
                    success_count += 1
                    found_alternative = True
                else:
                    conversion_failed.append({
                        'file': jpg_name,
                        'original': str(alt_path),
                        'error': error
                    })
                    found_alternative = True
                break
        
        if not found_alternative:
            no_gif_alternative.append(jpg_name)
            gif_not_found += 1

# 결과 출력
print("\n" + "=" * 70)
print("CONVERSION RESULTS")
print("=" * 70)
print(f"✅ Successfully converted: {success_count}")
print(f"⏭️  Already exists (JPG): {already_exists}")
print(f"❌ Source file not found: {gif_not_found}")
print(f"❌ Conversion failed: {len(conversion_failed)}")

# 변환 실패 상세
if conversion_failed:
    print("\n" + "-" * 70)
    print("CONVERSION FAILURES")
    print("-" * 70)
    for item in conversion_failed[:10]:  # 처음 10개만 표시
        print(f"  {item['file']}")
        print(f"    Error: {item['error']}")
    if len(conversion_failed) > 10:
        print(f"  ... and {len(conversion_failed) - 10} more")

# 소스 파일을 찾을 수 없는 경우
if no_gif_alternative:
    print("\n" + "-" * 70)
    print("SOURCE FILES NOT FOUND")
    print("-" * 70)
    if len(no_gif_alternative) <= 20:
        for fname in no_gif_alternative:
            print(f"  {fname}")
    else:
        for fname in no_gif_alternative[:20]:
            print(f"  {fname}")
        print(f"  ... and {len(no_gif_alternative) - 20} more")
    
    # 누락 파일 목록 저장
    missing_log = base_dir / "ocr_vqa_missing_sources.txt"
    with open(missing_log, 'w') as f:
        for fname in no_gif_alternative:
            f.write(f"{fname}\n")
    print(f"\n💾 Missing source files list saved to: {missing_log}")

print("\n" + "=" * 70)
print("🎯 SUMMARY")
print("=" * 70)

if success_count + already_exists == len(file_names):
    print("✅ ✅ ✅ ALL FILES READY! ✅ ✅ ✅")
elif gif_not_found > 0:
    print(f"⚠️  {gif_not_found} files are missing source images")
    print("These files will be skipped during training")
elif conversion_failed:
    print(f"⚠️  {len(conversion_failed)} files failed to convert")
    print("Review errors above")

Total files to convert: 176
OCR-VQA directory: /nas/datahub/llava-v1.5-instruct/ocr_vqa/images

Converting GIF to JPG


Processing:   0%|          | 0/176 [00:00<?, ?it/s]


CONVERSION RESULTS
✅ Successfully converted: 176
⏭️  Already exists (JPG): 0
❌ Source file not found: 0
❌ Conversion failed: 0

🎯 SUMMARY
✅ ✅ ✅ ALL FILES READY! ✅ ✅ ✅


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 39.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 58.8 MB/s  0:00:00
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/opt/venv/lib/python3.12/site-packages/widgetsnbextension'
Check the permissions.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/3 [widgetsnbextension]
